# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [117]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [118]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    assert len(file_path_list) == 30

/opt/dev_env


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [119]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [120]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [121]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(contact_points=['cassandra'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [122]:
# Create sparkify keyspace using a unique node
session.execute("""
                   CREATE KEYSPACE IF NOT EXISTS sparkify
                   WITH REPLICATION = { 
                          'class' : 'SimpleStrategy',
                          'replication_factor' : 1 }
                """)

#### Set Keyspace

In [123]:
# Set keyspace to sparkify
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [124]:
"""
Create the first table containing the historic of the music
The primare key was chosen regarding the fact thath sessionId and
itemInSession must be queried and that the compose key formed using
both columns results in a really unique primary key.
""" 
session.execute("""
                CREATE TABLE IF NOT EXISTS music_library (
                    sessionId int,
                    itemInSession int,
                    artist text,
                    length float,
                    song text,
                    PRIMARY KEY(sessionId, itemInSession))
                """)

"""
Create the second table regarding the songs listened by user
The primare key was chosen regarding the fact thath userId and
sessionId must be queried and that the compose key formed using
both columns results in a really unique primary key. Also
it makes sense to partition by user (so every session is clusered together)
and after that by the sessions instead of the other way around
"""
session.execute("""
                CREATE TABLE IF NOT EXISTS user_library (
                    userId int,
                    sessionId int,
                    itemInSession int,
                    artist text,
                    firstName text,
                    lastName text,
                    song text,
                    PRIMARY KEY(userId, sessionId, itemInSession))
                """)

"""
Create the third table regarding the songs listened by the users
In this case the primary key is composed of all the columns in orther to 
be fully unique and song as the partition key to make the queries
faster regarding the songs listened by.
"""
# Create third table
session.execute("""
                CREATE TABLE IF NOT EXISTS songs_per_user (
                    song text,
                    firstName text,
                    lastName text,
                    PRIMARY KEY(song, firstName, lastName))
                """)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [125]:
# Make the inserts in the at the same time, so the file is only looped once

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert rows @ music_library
        query_music = """
                         INSERT INTO music_library (artist, itemInSession, length, sessionId, song)
                         VALUES (%s, %s, %s, %s, %s);
                      """
        session.execute(query_music, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9]))  
        
        # Insert rows @ user_library
        query_user = """
                        INSERT INTO user_library (artist, firstName, itemInSession, lastName, sessionId, song, userId)
                        VALUES (%s, %s, %s, %s, %s, %s, %s);
                     """
        session.execute(query_user, (line[0], line[1], int(line[3]), line[4], int(line[8]), line[9], int(line[10])))
        
        # Insert rows @ songs_per_user
        query_songs = """
                        INSERT INTO songs_per_user (firstName, lastName, song)
                        VALUES (%s, %s, %s);
                      """
        session.execute(query_songs, (line[1], line[4], line[9]))

In [126]:
# Retieve the artist, song and song length regarding sessionId 338 and itemSession 4
query_one = """
               SELECT artist, song, length FROM music_library
               WHERE sessionId = 338 AND itemInSession = 4;
            """

#### Do a SELECT to verify that the data have been inserted into each table

In [127]:
# Verify select of query_one
results = session.execute(query_one)
for idx, res in enumerate(results):
    print(f'{idx}. Row ->  Artist: {res.artist}, Song: {res.song}, Length: {res.length}')

0. Row ->  Artist: Faithless, Song: Music Matters (Mark Knight Dub), Length: 495.30731201171875


### Query for the second table

In [128]:
# Second query
query_two = """
               SELECT artist, song, firstName, lastName FROM user_library
               WHERE userId = 10 AND sessionId = 182;
            """

In [129]:
# Verify select of query_two
results = session.execute(query_two)
for idx, res in enumerate(results):
    user = f'{res.firstname} {res.lastname}'
    print(f'{idx}. Row ->  Artist: {res.artist}, Song: {res.song}, User: {user}')

0. Row ->  Artist: Down To The Bone, Song: Keep On Keepin' On, User: Sylvie Cruz
1. Row ->  Artist: Three Drives, Song: Greece 2000, User: Sylvie Cruz
2. Row ->  Artist: Sebastien Tellier, Song: Kilometer, User: Sylvie Cruz
3. Row ->  Artist: Lonnie Gordon, Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), User: Sylvie Cruz


### Query the third table

In [130]:
# Third query
query_three = """
               SELECT firstName, lastName FROM songs_per_user
               WHERE song = 'All Hands Against His Own';
            """

In [131]:
# Verify select of query_three
results = session.execute(query_three)
for idx, res in enumerate(results):
    user = f'{res.firstname} {res.lastname}'
    print(f'{idx}. Row -> User: {user}')

0. Row -> User: Jacqueline Lynch
1. Row -> User: Sara Johnson
2. Row -> User: Tegan Levine


### Drop the tables before closing out the sessions

In [132]:
session.execute('DROP TABLE IF EXISTS music_library;')
session.execute('DROP TABLE IF EXISTS user_library;')
session.execute('DROP TABLE IF EXISTS song_per_user;')

### Close the session and cluster connection¶

In [133]:
session.shutdown()
cluster.shutdown()